$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Reinforcement Learning Project for League of Legends Champion Select

by Julien Pecquet, December 11th, 2018

# Introduction

League of Legends (A.K.A. LoL or League) is an Multiplayer Online Battle Arena (MOBA) game, where teams of 5 fight each other with the goal of destroying the opposing teams nexus (home base). [The League of Legends wikipedia](https://en.wikipedia.org/wiki/League_of_Legends) notes that in September 2016, the company who owns LoL, Riot Games, estimated that there are over 100 million active players each month. Currently, the 2018 World Championship is ongoing, which featured teams from North America, Europe, Latin America, South Korea, China, Taiwan, and many more. Last year's championship boasted 60 million unique viewers and a total prize pool of over 4 million USD. Needless to say, the pay off for succeeding in the League of Legends esports scene is enormous, and noticed by a huge population. This is one of my motivations for solving this problem.
    
While it would be incredible to focus on a program who could play the real-time game against real people, this would be an incredibly deep problem requiring much more resources. This is in fact the part of the game which most people think of when they think about LoL. This being said, the portion of the game I want to focus on is similar to a card game. It is known as Champion Select. Before players can get to the MOBA action, they go through a turn-based game which takes about 5 minutes to decide which champions they get to play. It has many formats and I want to explore them all to see what my implementation will think is best within each format. I also am very excited to compare what my program will value as opposed to what players come up in different countries and different ranks, to my own ideas, and also to professional play. 

Before I dive into the specifics, let's think for a moment about the space complexity of this problem given the different drafts. There are (as of December 5th, 2018) 142 Champions in the League. Basic rules of league's interactive draft state that you can only have unique champions in the pick/bans. There are 5 bans per team, 5 champions per team. So there are (142 choose 20)=1.12 * 10^24 possible board states.<br>

Now, how do we cut this number up so that this problem is doable? We can potentially completely remove ban's from the game, when training. This would mean that the problem goes to (142 choose 10)=6.64 * 10^14, significantly smaller! Something else to note is that not every champion can play in every role. Some champions only play in 1 role, so once that role is chosen they no longer are a choice for that game state. Implied in this statement is the fact that each team needs 1 and only 1 of each role. From data gathered using methods defined below, there are 107 top laners, 86 junglers, 101 mid laners, 84 duo_carries, 83 duo_supports. This drastically reduces the space of the problem to be less than before. Perhaps it would work something like this:

In [1]:
((142)*(141)*(140)*(139)*(138)*(137)*(136)*(135)*(107)*(107))/(10*9*8*7*6*5*4*3*2*(142-10))

3232609275050.784

My reasoning is that the first 4 choices per team (8 choices total) will, at worst, still allow choices of every remaining champion. But once you get to your last choice, you only have one role. I'm assuming the worst again, in that the last role picked is Top. 

How can we make the players pick things more randomly in a way that reflects a real game? We could have a mechanism for banning either a high general win-rate champ, or a random champ. This would mean that the most overpowered compositions would be found less often, and we wouldnt adhere to the meta quite as much. Making bans random would also reduce the amount of choices by a very significant amount. We can also make the opponent player randomly pick champions in order to increase processing time.

In [2]:
((132)*(131)*(130)*(129)*(128)*(127)*(126)*(125)*(107)*(107))/(10*9*8*7*6*5*4*3*2*(132-10))

1920070742519.672

Now, before you continue reading... this problem space is still just absolutely massive. My naivete brought me into this project thinking "it won't be so bad, what could go wrong?".
Well I'll tell you right now, the experiment failed because the problem is just too large to train the AI in a decent amount of time given the methods, organization, etc that I came up with. I would love to continue this problem and figure out where and why my code is so slow. As it stands, this is what I've got. I still think the project was very interesting and worth the read! Perhaps it is possible to see just where I went wrong. Hopefully, I didn't go wrong when I made the choice to pick this problem.

# Methods

To start the project, I wanted to make sure I understood how to use the API's in question before attempting to set up anything with the actual artificial intelligence algorithms. To do this, I ran a few tests with the API's and learned about the bits of each JSON that I was retrieving.
The services I am using are:
   * Champion.gg: Documentation found here: http://api.champion.gg/docs/#api-Champions
        * This tool is my primary one, which will get me an enormous amount of statistics for different champions in different roles with different other champions as their teammates or enemies. It has *almost* everything I need for the project.
   * Riot Games' Data Dragon: Documentation found here: https://developer.riotgames.com/static-data.html
        * This tool will allow me to do a great number of things to make my software easily usable (for me) and to make the output pretty! This service allows me to get images of champions, abilities, etc and more importantly for development it will allow me to get champion's descriptions by id (given to me from champion.gg requests).

First, let's import some useful stuff, as we always do.

In [3]:
import requests as req
import urllib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import copy

This is an example query for [Champion.gg](https://champion.gg/)

In [4]:
api_key = 'e29bf7c5e411c43e2db51ceb2255e3d1' #This will be used in many functions down the road, so I declare it now.
championgg = req.get("http://api.champion.gg/v2/champions?&limit=500&api_key="+api_key).json()

This is an example query for [Data Dragon](https://developer.riotgames.com/static-data.html)

In [6]:
dd = req.get("http://ddragon.leagueoflegends.com/cdn/8.24.1/data/en_US/champion.json").json()

In [312]:
req.get("http://ddragon.leagueoflegends.com/cdn/8.24.1/data/en_US/champion.json").json()

{'type': 'champion',
 'format': 'standAloneComplex',
 'version': '8.24.1',
 'data': {'Aatrox': {'version': '8.24.1',
   'id': 'Aatrox',
   'key': '266',
   'name': 'Aatrox',
   'title': 'the Darkin Blade',
   'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...',
   'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4},
   'image': {'full': 'Aatrox.png',
    'sprite': 'champion0.png',
    'group': 'champion',
    'x': 0,
    'y': 0,
    'w': 48,
    'h': 48},
   'tags': ['Fighter', 'Tank'],
   'partype': 'Blood Well',
   'stats': {'hp': 580,
    'hpperlevel': 80,
    'mp': 0,
    'mpperlevel': 0,
    'movespeed': 345,
    'armor': 33,
    'armorperlevel': 3.25,
    'spellblock': 32.1,
    'spellblockperlevel': 1.25,
    'attackrange': 175,
    'hpregen': 5,
    'hpr

Once I understood how to use these amazing assets, I began to think about design. After fiddling with the different queries these API's accept, it quickly became clear that I wouldn't want to write long complicated queries very often as it would get tedious, be error prone, and ultimately hard to digest for a reader. Besides this, there is a limited number of requests per unit time. At least for Champion.gg, this is a beta API which is completely free and open to anyone, so they simply don't have the infrastructure of a million queries whenever I want to train. More importantly for me, it will lock my key out (temporarily or permanently, I am not sure). Also, it would be more time and space efficient to only have to query the minimum amount of times, and to hold this information on my machine.

Something else I wanted to take into account was applying object-oriented design to make it simpler to read, write, and manage. If something went wrong, or if I wanted to add more functionality to a piece of my code, or perhaps if something worked very differently than I anticipated, I know that I would particularly want encapsulation implemented. This is reflected in my design below.

I borrowed the reinforcement learning code from class, looking at the Maze and particularly Tic-Tac-Toe code much like in one of our assignments. I also, of course, reviewed my own assignment code to remember what alterations I made to the courses code to make it work on a new problem. That being said, this problem is closer to Tic-Tac-Toe than it is to Maze or Hanoi, since it is a two-player game.

## Class: Champion
![alt text](http://ddragon.leagueoflegends.com/cdn/img/champion/splash/Aatrox_0.jpg)

This class is key to my design. It holds important information regarding a specific champion. When we are training the AI in Champion Select, it will need access to this information every time it gets to decides which champion to pick (i.e. when it is not a random choice). Within the realm of reinforcement learning, think of this class as each of the choices. For each champion not yet chosen at each node, there is a branch in the tree of choices which constitutes picking that champion.<br>
The data it holds looks like this:
   - `name`: The human-recognizable identifier for the Champion, such as "Aatrox" as shown above.
   - `id`: The consistent identifier for grabbing a champion from Data Dragon or Champion.gg, such as '266'.
   - `winrate`: The averaged winrate over all of the Champion's roles.
   - `roles`: Any one champion can have any combination of the following entires of this list as their roles: 
    - `[TOP, JUNGLE, MIDDLE, DUO_CARRY, DUO_SUPPORT, SYNERGY, ADCSUPPORT]`.<br>
     - `TOP, JUNGLE, MIDDLE, DUO_CARRY, DUO_SUPPORT`: describe what is known as a "lane assignment". In the community, these are what people think of when they think of roles in a composition within League of Legends. In this program, I will limit each team of 5 to one of each within this list. Can't make a team with 5 `DUO_SUPPORT`, for example. 
     - `SYNERGY`: refers to two champions played on the same team, either playing in any role as long as it is not the same role. 
     - `ADCSUPPORT`: refers to part of the `DUO_SUPPORT`/`DUO_CARRY` relationship, and can be thought of as `BOTTOM SYNERGY` since `DUO_SUPPORT` and `DUO_CARRY` both play in the bottom lane.
   - `matchups`: A list in the form 
    -`[ROLE: {ENEMY/ALLY: WINRATE WITH/AGAINST}, ...]`<br> 
    This structure describes how `Champion A` relates to `Champion B` in terms of `winrate`. For every entry in the Champion's `roles` list, there is an entry in this list which goes more in depth. This could, for example, be used to find the winrate between Aatrox and Alistar, or the winrate when Aatrox and Alistar are on the same team.<br>

In [7]:
#Given an id number, and Data Dragon, returns all champion info
def getChampInfoById(champId, dataDragon):
    for key, value in dataDragon['data'].items():
        if int(value['key']) == int(champId):
            return value
    return None

#[ROLE: {<ENEMY/ALLY Champion>: <Winrate AGAINST/WITH>}, ...}, ...]
def getAllMatchups(champId, championgg, dataDragon, limit=10):
    #Every matchup
    champId = str(champId)
    matchups = req.get("http://api.champion.gg/v2/champions/"+champId+"/matchups?&limit=500&api_key="+api_key).json()

    #All champions: {64: 'LeeSin',...}
    allChamps = getAllChamps(championgg, dataDragon)

    #Filter the matchups, limiting it to only *limit* games as bottom threshold.
    #For example if limit=10, trim off all matchups with less than 10 games played.
    fm = filterMatchups(matchups, limit)
    all_roles = ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT', 'SYNERGY', 'ADCSUPPORT']
    all_matchups = {}

    #Iterate through each role type
    for role in all_roles:
        #Role_fm = role with filtered matchups. Looks at Lee Sin Jungle vs all(x) Jungle.
        role_fm = [x for i, x in enumerate(fm) if fm[i]['_id']['role'] == role]
        #print(role)
        #print(role_fm)

        #If there is at least one game...
        if role_fm:

            role_matchup = []
            for i, x in enumerate(role_fm):
                #Figure out which champ_id is our champ, and which is the enemy, get the winrate AGAINST:
                if int(x['_id']['champ2_id']) != int(champId):
                    enemyChamp = allChamps[x['_id']['champ2_id']]
                    enemyChampId = x['_id']['champ2_id']
                    winrate = role_fm[i]['champ1']['winrate']
                else:
                    enemyChamp = allChamps[x['_id']['champ1_id']]
                    enemyChampId = x['_id']['champ1_id']
                    winrate = role_fm[i]['champ2']['winrate']
                #Add this information to a dictionary
                matchup = {}
                matchup[enemyChamp] = winrate

                #Add it to the collection of role_matchups
                role_matchup.append(matchup)
            #Add this role_matchup to all matchups
            all_matchups[role] = role_matchup
    return all_matchups

#Given champion.gg and Data Dragon champion data jsons, returns a dictionary of {id : champion name}
def getAllChamps(championgg, datadragon):
    champs = {}

    #Iterate through all champions in championgg
    for i in range(len(championgg)):
        championggKey = championgg[i]['_id']['championId']

        #Iterate through all champions in datadragon
        for key, value in datadragon['data'].items():

            #If the champion id's match...
            if int(value['key']) == int(championggKey):
                #... add it to the list, and go to next
                champs[championggKey] = key
                break
    return champs

#Given a name, and Data Dragon, returns the champion's id
def getChampId(champName, dataDragon):
    return dd['data'][champName]['key']

def getWinrate(matchups):
    winrate = 0
    total = 0
    for role in matchups:
        for matchup in matchups[role]:
            winrate += list(matchup.values())[0]
            total += 1
    return winrate/total

def getWinrate(matchups, role):
    roleMatchups = matchups[role]
    winrate = 0
    total = 0
    for matchup in roleMatchups:
        winrate += list(matchup.values())[0]
        total += 1
    if total == 0:
        return .5
    return winrate/total

#Given a set of matchups, filter the set by how big the data size is, defaulting at limit=100.
def filterMatchups(matchups, limit=100):
    return [matchup for matchup in matchups if matchup['count'] >= limit]

In [8]:
#Holds information about one champion
class Champion:    
    def showIcon(self):
        plt.imshow(img)
        plt.show()
        
    #Given a lane, gives the best match up aka the champ this champ has the highest win rate against/with.
    def getBestMatchup(self, lane):
        winrates = self.matchups[lane]
        bestWinrate = 0
        bestIndex = -1
        for i in range(len(winrates)):
            win = list(winrates[i].values())[0]
            #print(win)
            if win > bestWinrate:
                bestWinrate = win
                bestIndex = i
        return winrates[bestIndex]


    def getWorstMatchup(self, lane):
        winrates = self.matchups[lane]
        bestWinrate = 1
        bestIndex = -1
        for i in range(len(winrates)):
            win = list(winrates[i].values())[0]
            #print(win)
            if win < bestWinrate:
                bestWinrate = win
                bestIndex = i
        return winrates[bestIndex]
    
    def __init__(self, championId, championgg, dataDragon):
        self.dataDragon = getChampInfoById(championId, dataDragon)
        self.championgg = championgg                                  #TODO: Make League filter this field.           
        
        #Matchups
        self.matchups = getAllMatchups(championId, championgg, dataDragon)
        
        #Roles in matchups that are not SYNERGY or ADCSUPPORT
        self.roles = [role for role in list(getAllMatchups(championId, championgg, dd).keys()) \
                      if role != 'SYNERGY' and role != 'ADCSUPPORT']
        
        #Any non-zero positive integer found in the Data Dragon champion data
        self.id   = championId                   
        
        #Irelia, LeeSin, Aatrox...
        self.name = self.dataDragon['id']  
        
        #Image information
        #self.icon = mpimg.imread('http://ddragon.leagueoflegends.com/cdn/6.24.1/img/champion/' + self.dataDragon['image']['full'])
        
    def __repr__(self):
        return str(self.name) + ", " + str(self.id) + ": " + str(self.roles)
    
    def __eq__(self, other):
        return self.name == other.name

For whatever reason, either Jupyter or Python 3 didnt want me to declare the Champion methods within class Champion. I'm not experienced enough with Python to know how to fix it so I just put them adjacent to the class.

A quick example:

In [9]:
Champion(64, championgg, dd)

LeeSin, 64: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']

## Class: League
League of Legends boasts 142 unique Champions. This class is responsible for creating a catalogue of those champions. It will query Champion.gg and Data Dragon and create a *list of Champions* from the returned data. When we are picking champions, we will need a list to look at to inform our decision. 
![alt text](https://i.imgur.com/ptIRC1C.png)
Image taken from the [League of Legends Wiki](http://leagueoflegends.wikia.com/wiki/League_of_Legends_Wiki)

In [10]:
#Creates a collection of every Champion.
#It accomplishes this by querying Data Dragon and Champion.gg for the requested patch, using the provided key.
class League:
    #A constructor if you know nothing, or know patch and/or api_key
    def __init__(self, patch="8.24.1", api_key="e29bf7c5e411c43e2db51ceb2255e3d1"):
        self.championgg = req.get("http://api.champion.gg/v2/champions?&limit=500&api_key="+api_key).json()
        self.dd = req.get("http://ddragon.leagueoflegends.com/cdn/"+patch+"/data/en_US/champion.json").json()
        self.champions = [Champion(getChampId(name, dd), championgg, dd) for name in dd['data']]
        
    def getChampsInRole(self, role):
        return [champ for champ in self.champions if role in champ.roles]
    
    def getListOfChamps(self):
        return [champ for champ in self.champions]

Here is an introduction to using the League class. You can see the way in which things are organized, and can perhaps begin to think about how I want to use this collection I have created.

In [11]:
L = League()

We can retrieve the complete list of champions. Also if you want to read the names of the champions (some are quite interesting) this is the place to do it!

In [12]:
L.getListOfChamps()

[Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Ahri, 103: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Akali, 84: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY'],
 Alistar, 12: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_SUPPORT'],
 Amumu, 32: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_SUPPORT'],
 Anivia, 34: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Annie, 1: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Ashe, 22: ['MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 AurelionSol, 136: ['MIDDLE'],
 Azir, 268: ['TOP', 'MIDDLE', 'DUO_CARRY'],
 Bard, 432: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Blitzcrank, 53: ['JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Brand, 63: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Braum, 201: ['DUO_SUPPORT'],
 Caitlyn, 51: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Camille, 164: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_SUPPORT'],
 Cassiopeia, 69: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Chogath, 31: ['TOP', 'JUNGLE', 'MIDDLE

We want to learn more about the champion Aatrox, specifically his winrates in the top lane:

In [14]:
Aatrox = listOfChamps[0]
print(Aatrox.getBestMatchup('TOP'))
print(Aatrox.getWorstMatchup('TOP'))

{'Anivia': 0.7142857142857143}
{'Shyvana': 0.3488372093023256}


From this information, if we see Aatrox picked, we could definitely consider picking Anivia, and really shouldn't pick Shyvana. It is important to note that if this code is ran again, these champion names will almost certainly be different. Champion.gg updates their statistics twice daily.

# Class: ChampionSelect (The Game)
As of right now, the functions which would reside in this class are global methods. I was having difficulties getting Jupyter to assemble classes exactly how I expected. So to remove this complication, I left the following details out of my final deliverable.
* Holds the code to calculate ending win chance of a given composition
* Holds the code to `play a Game` given `n` Selectors and a ruleset.
 * `Ruleset`:  will only be `BLIND` for now. But could also be `DRAFT_SOLO, DRAFT_RANKED, DRAFT_LCS, ARAM, NB, TT`
 * `Selector=1`: Creates a random selector to play against.
 * `Selector=2`: Plays the two against each other and returns the comps and win chance for each comp.
* Holds the code to reinforce a Selector: the borrowed Reinforcement Learning algorithm from class.
* `usage`: `ChampionSelect.play(Selector1, Select2, 'BLIND', numGames)`

### Get valid moves (bans and picks)

When asking what valid moves we have, we need to take a few things into account:
 * What are the current compositions? In Champ Select, there are two "sides" of the board. So we need to know the ally side and enemy side rather than simply the board and pieces on it. It is mostly just more clear to split them up.
 * What are the banned champions? Need to remove these from the dict to be returned.
 * What does a composition look like? It's going to look something like this: `{'Aatrox': 'TOP', 'Zoe': 'MIDDLE'}` for a state where one comp already has picked twice. 

In [186]:
def getValidChampMoves(league, comp1, comp2, bans=[]):
    champList = L.getListOfChamps()
    
    #Get the remaining available roles:
    allRoles = ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']
    takenRoles = list(comp1.keys())
    availableRoles = [x for x in allRoles if x not in takenRoles]
    
    #Get the remaining available champions:
    takenChamps = list(comp1.values()) + list(comp2.values()) + bans
    availableChamps = [x for x in champList if x not in takenChamps]
    return [{role: champ} for champ in availableChamps for role in champ.roles if role in availableRoles]

In [187]:
Aatrox = Champion(getChampId('Aatrox', dd), championgg, dd)
Ahri = Champion(getChampId('Ahri', dd), championgg, dd)
Zoe = Champion(getChampId('Zoe', dd), championgg, dd)
Zyra = Champion(getChampId('Zyra', dd), championgg, dd)

In [188]:
comp1 = {'TOP': Aatrox, 'MIDDLE': Zoe}
comp2 = {'MIDDLE': Ahri}
bans = [Zyra]

In [189]:
validChampMoves = getValidChampMoves(L, comp1, comp2, bans)
validChampMoves[:2]

[{'JUNGLE': Akali, 84: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY']},
 {'DUO_CARRY': Akali, 84: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY']}]

As you can see, we can still choose Akali even though TOP and MIDDLE have both been taken. It shows that we can put Akali in the JUNGLE or DUO_CARRY positions. Also note that Aatrox is no longer the first in the list of champs, because we have chosen him, and Ahri is also gone because the enemy chose her!

In [190]:
validChampMoves = getValidChampMoves(L, {}, {})
validChampMoves[:5]

[{'TOP': Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']},
 {'JUNGLE': Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']},
 {'MIDDLE': Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']},
 {'DUO_CARRY': Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']},
 {'DUO_SUPPORT': Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']}]

At the start of a game, when there are no bans or champs picked, we can still choose Aatrox in any position. Working as intended!

Now we need a way to ban champs that have not already been banned. Note that this function is only called on a per-team basis in some game modes. So both team could, theoretically, ban the exact same 5 champions. Also, "None" is a valid ban.

In [191]:
def getValidBanMoves(league, bans=[]):
    champList = L.getListOfChamps()
        
    #Get the remaining available champions:
    takenChamps = bans
    availableChamps = [x for x in champList if x.name not in takenChamps]
    return [champ for champ in availableChamps]
    #return [champ for champ in availableChamps for role in champ.roles if role in availableRoles] + ['None']

In [192]:
getValidBanMoves(L, ['Aatrox'])

[Ahri, 103: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Akali, 84: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY'],
 Alistar, 12: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_SUPPORT'],
 Amumu, 32: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_SUPPORT'],
 Anivia, 34: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Annie, 1: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Ashe, 22: ['MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 AurelionSol, 136: ['MIDDLE'],
 Azir, 268: ['TOP', 'MIDDLE', 'DUO_CARRY'],
 Bard, 432: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Blitzcrank, 53: ['JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Brand, 63: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Braum, 201: ['DUO_SUPPORT'],
 Caitlyn, 51: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Camille, 164: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_SUPPORT'],
 Cassiopeia, 69: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Chogath, 31: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 Corki, 42: ['TOP', 'MIDDLE', 'DUO_CARR

### Make move:

In [193]:
#A move looks like: {'JUNGLE', Akali}
def makeChampMove(league, move, comp1, comp2, bans=[]):
    #Make sure the move is valid:
    if move not in getValidChampMoves(league, comp1, comp2, bans):
        #If it is an invalid move, return the ally composition unchanged.
        return comp1
    
    #Set up a new copy of the comp
    newComp = copy.deepcopy(comp1)
    
    #Make the move: Add the move to the comp
    newComp[list(move.keys())[0]] = list(move.values())[0]
    
    #Return the new comp!
    return newComp

In [194]:
Akali = Champion(getChampId('Akali', dd), championgg, dd)
comp1 = {'TOP': Aatrox, 'MIDDLE': Zoe}
comp2 = {'MIDDLE': Ahri}
bans = [Zyra]

In [195]:
move = {'JUNGLE': Akali}
newComp1 = makeChampMove(L, move, comp1, comp2, bans)

In [196]:
comp1

{'TOP': Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 'MIDDLE': Zoe, 142: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']}

The champion, Akali is added to the JUNGLE role.

In [197]:
newComp1

{'TOP': Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 'MIDDLE': Zoe, 142: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 'JUNGLE': Akali, 84: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY']}

In [198]:
comp1 = {'TOP': Aatrox, 'MIDDLE': Zoe}
comp2 = {'MIDDLE': Ahri}
bans = [Zyra]
move = {'MIDDLE': Akali}
newComp1 = makeChampMove(L, move, comp1, comp2, bans)

In [199]:
comp1

{'TOP': Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 'MIDDLE': Zoe, 142: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']}

The champion Akali is NOT added to middle, as middle is already taken by Zoe.

In [200]:
newComp1

{'TOP': Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
 'MIDDLE': Zoe, 142: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']}

### Winner

Before we can calculate the winner, we need to come up with an equation for win chance. I think it is in my best interest to make this overly simple: I will add up the win rates for each champion in their current matchup and divide by 5. If the matchup does not exist, I will add the average winrate for the role in place of the matchup winrate. If I improve on this program later, I will add in synergy and adcsupport for roles, as these will impact the choices made. For example, in the 2018 World's championship, picking Lucian when your team had Braum increased a team's win rate, but decreased it without Braum (either stolen, banned, or never picked).

In [201]:
def calcWinChance(league, comp1, comp2):
    #Gather comp1,2 info into separate structures
    roles = list(comp1.keys())
    champs = list(comp1.values())
    badRoles = list(comp2.keys())
    badChamps = list(comp2.values())

    winrate = 0
    for i, champ in enumerate(champs):
        matchups = champ.matchups[roles[i]]
        #print("Checking: " + champ.name + " " + roles[i])
        #print("Looking for: " + badChamps[i].name + " " + badRoles[i])
        found = False
        for matchup in matchups:
            #print(list(matchup.keys())[0])
            if list(matchup.keys())[0] == badChamps[i].name:
                #print("match!")
                #print(matchup)
                found = True
                winrate += list(matchup.values())[0]
                break
        #No data on the matchup
        if not found:
            #print(champ)
            #print(matchups)
            winrate += getWinrate(champ.matchups, roles[i])
    return winrate/5

In [202]:
#Comp 1
Aatrox = Champion(getChampId('Aatrox', dd), championgg, dd) #Top
Alistar = Champion(getChampId('Alistar', dd), championgg, dd) #Jungle
Ahri = Champion(getChampId('Ahri', dd), championgg, dd) #Middle
Anivia = Champion(getChampId('Anivia', dd), championgg, dd) #Carry
Annie = Champion(getChampId('Annie', dd), championgg, dd) #Support

#Comp 2
Zyra = Champion(getChampId('Zyra', dd), championgg, dd) #Top
Zed = Champion(getChampId('Zed', dd), championgg, dd) #Jungle
Zoe = Champion(getChampId('Zoe', dd), championgg, dd) #Middle
Ziggs = Champion(getChampId('Ziggs', dd), championgg, dd) #Carry
Zilean = Champion(getChampId('Zilean', dd), championgg, dd) #Support

In [203]:
comp1 = {'TOP': Aatrox, 'JUNGLE': Alistar, 'MIDDLE': Ahri, 'DUO_CARRY': Anivia, 'DUO_SUPPORT': Annie}
comp2 = {'TOP': Zyra, 'JUNGLE': Zed, 'MIDDLE': Zoe, 'DUO_CARRY': Ziggs, 'DUO_SUPPORT': Zilean}

In [204]:
calcWinChance(L, comp1, comp2)

0.5226591419417862

The way I will check if there is a winner is by calculating an average of the winrates of the champions in the roles they've been assigned for both comps, then comparing them. If the game is over, and blue wins, return True, True. If game is over and red wins, return True, False. Else, return False, False.

In [205]:
def winner(league, state):
    #Game is not finished yet
    if len(state['blue']) != 5 or len(state['red']) != 5:
        return False, False
    
    #Return True because game is over, then return whether blue team won or not
    return True, calcWinChance(league, state['blue'], state['red']) >= calcWinChance(league, state['red'], state['blue'])

In [297]:
comp1 = {'TOP': Aatrox, 'JUNGLE': Alistar, 'MIDDLE': Ahri, 'DUO_CARRY': Anivia, 'DUO_SUPPORT': Annie}
comp2 = {'TOP': Zyra, 'JUNGLE': Zed, 'MIDDLE': Zoe, 'DUO_CARRY': Ziggs, 'DUO_SUPPORT': Zilean}
state = {'blue': comp1, 'red': comp2}
winner(L, state)[1]

True

In [207]:
comp1Win = calcWinChance(L, comp1, comp2)
comp2Win = calcWinChance(L, comp2, comp1)
str(comp1Win) + " vs " + str(comp2Win)

'0.5226591419417862 vs 0.5004665934058649'

winner should return true, as comp1 has better chances.

In [209]:
winner(L, state)

(True, True)

### Print state

This method is going to give us a way to see the champs in the comp. My idea was to make it print something very pretty with the icons for each champ with their names and some stats, but time only permitted a basic text print function.

In [295]:
def printState(comp1, comp2, bans):
    #Print ban board
    banList = []
    for ban in bans:
        banList.append(ban.name)
    print("Bans: " + str(banList))
    
    #Print champ board
    roles1 = list(comp1.keys())
    champs1 = list(comp1.values())
    roles2 = list(comp2.keys())
    champs2 = list(comp2.values())
    print("----------------------")
    if len(roles1) == 5 and len(roles2) == 5:
        for i in range(5):
            print(roles1[i])
            print(champs1[i].name + " vs " + champs2[i].name)
            print()
    state = {'blue': comp1, 'red': comp2}
    if winner(L, state):
        print("Left team has " + "{0:.2f}".format(calcWinChance(L, comp1, comp2)) + " chance of a win.")
    else:
        print("Right team has " + "{0:.2f}".format(calcWinChance(L, comp2, comp1)) + " chance of a win.")
    print("----------------------")

In [211]:
bans

[Zyra, 143: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']]

In [298]:
printState(comp1, comp2, bans)

Bans: ['Zyra']
----------------------
TOP
Aatrox vs Zyra

JUNGLE
Alistar vs Zed

MIDDLE
Ahri vs Zoe

DUO_CARRY
Anivia vs Ziggs

DUO_SUPPORT
Annie vs Zilean

Left team has 0.52 chance of a win.
----------------------


### stateMoveTuple

In [224]:
def stateMoveTuple(state, move):
    blue = state['blue']
    blueRoles = list(blue.keys())
    blueChampions = list(blue.values())
    #print("bluechamps: " + str(blueChampions))
    blueChampions = [champ.name for champ in blueChampions]
    
    red = state['red']
    redRoles = list(red.keys())
    redChampions = list(red.values())
    redChampions = [champ.name for champ in redChampions]
    
    blueTuple = tuple(blueRoles), tuple(blueChampions)
    redTuple = tuple(redRoles), tuple(redChampions)
    return blueTuple, redTuple

In [215]:
comp1 = {'TOP': Aatrox, 'JUNGLE': Alistar, 'MIDDLE': Ahri, 'DUO_CARRY': Anivia, 'DUO_SUPPORT': Annie}
comp2 = {'TOP': Zyra, 'JUNGLE': Zed, 'MIDDLE': Zoe, 'DUO_CARRY': Ziggs, 'DUO_SUPPORT': Zilean}
state = {'blue': comp1, 'red': comp2}
stateMoveTuple(state, move)

bluechamps: [Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'], Alistar, 12: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_SUPPORT'], Ahri, 103: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'], Anivia, 34: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'], Annie, 1: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']]


((('TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'),
  ('Aatrox', 'Alistar', 'Ahri', 'Anivia', 'Annie')),
 (('TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'),
  ('Zyra', 'Zed', 'Zoe', 'Ziggs', 'Zilean')))

## Reinforcement Learning Algorithm

Borrowed from class, of course, then modified to fit my implementation of my move-related code.

   - `trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF)`: train the Q function for number of repetitions, decaying epsilon at start of each repetition. Returns Q and list or array of number of steps to reach goal for each repetition.

   - `testQ(Q, maxSteps, validMovesF, makeMoveF)`: without updating Q, use Q to find greedy action each step until goal is found. Return path of states.
   
   - `epsilonGreedy(epsilon, Q, state)`: Takes an epsilon, Q dictionary and state. With these, it will return either a random move or the first greedy move. If epsilon is below .5, higher chances for random move and vise-versa.

In [223]:
def epsilonChampGreedy(epsilon, Q, league, comp1, comp2, bans=[]):
    validMoves = getValidChampMoves(league, comp1, comp2, bans)
    if np.random.uniform() < epsilon:
        # Random Move
        #print(" RANDOM NUMBER "+"\n" + str(np.random.choice(len(validMoves))))
        #print(" VALID MOVES "+"\n" + str(validMoves))
        return validMoves[np.random.choice(len(validMoves))]
    else:
        # Greedy Move
        Qs = np.array([Q.get(stateMoveTuple(state, move), 0) for move in validMoves]) 
        return validMoves[ np.argmax(Qs) ]

### Train Q

The way I wrote my trainQ function was such that it could be expandable in order to include a ban phase (I omitted it to reduce the complexity) and structured it so that it could be easily altered to have a "stupid" player 2, or one that is learning to play against player 1. Note that for testQ, we strictly stick with player 1. The way this works is that you take turns by draft pick rules (you can read which turns are blue, which are red) and each player picks until there is an end state. When there is an end state (winner check), we then check who is the winner and if it is blue, we reinforce by 1. On the other hand, we reinforce 0 for red. Other than that, this code is incredibly similar to what I turned in for my A4 assignment, combined with the tic-tac-toe code and some rules for playing Champion Select.

In [ ]:
def trainQ(nRepetitions, learningRate, epsilonDecayFactor, league, validChampMovesF, validBanMovesF, makeChampMoveF):
    rho = learningRate                    #I left this assignment here because rho is more compact.
    epsilonDecayRate = epsilonDecayFactor #Not sure why these are named differently, just reused assignment.
    epsilon = 1.0                         #Start at 1 and decay
    graphics = False                      #Graphics may be used eventually, leaving here just in case.
    showMoves = not graphics              #Similarly, I may use this functionality, so leaving here.

    outcomes = np.zeros(nRepetitions)
    epsilons = np.zeros(nRepetitions)
    Q = {}

    if graphics:
        fig = plt.figure(figsize=(10,10))

    #Play a game each repetition.
    for repetitions in range(nRepetitions):
        print("Game: " + str(repetitions))
        epsilon *= epsilonDecayRate
        epsilons[repetitions] = epsilon
        step = 0
        
        #Ban phase
        #Grab empty bans
        bans = [] 
        banDone = False
        
        
        #Pick phase
        #Grab empty comp1, comp2, initialize turn start
        comp1 = {}
        comp2 = {}
        state = {'blue': comp1, 'red': comp2}
        turn = 1
        pickDone = False
        while not pickDone:        
            step += 1
            #print(turn)
            #If blue/comp1 turn [1,4,5,8,9]
            if turn == 1 or turn == 4 or turn == 5 or turn == 8 or turn == 9:
            
                # Make a move
                move = epsilonChampGreedy(epsilon, Q, league, state['blue'], state['red'], bans)
                newComp = makeChampMove(league, move, state['blue'], state['red'], bans=[])
                newState = copy.deepcopy(state)
                newState['blue'] = newComp #Update the blue comp
                if stateMoveTuple(state, move) not in Q:
                    Q[stateMoveTuple(state, move)] = 0  # initial Q value for new state, move
                turn += 1
                
            
            #If red/comp2 turn [2,3,6,7,10]
            elif turn == 2 or turn == 3 or turn == 6 or turn == 7 or turn == 10: 
            
                # Make a move
                move = epsilonChampGreedy(epsilon, Q, league, state['red'], state['blue'], bans)
                newComp = makeChampMove(league, move, state['red'], state['blue'], bans=[])
                newState = copy.deepcopy(state)
                newState['red'] = newComp #Update the bredlue comp
                if stateMoveTuple(state, move) not in Q:
                    Q[stateMoveTuple(state, move)] = 0  # initial Q value for new state, move
                turn += 1
                
            #print(stateMoveTuple(newState, move))
            
            #Is the game over?
            if winner(league, newState)[0]:
                pickDone = True
                outcomes[repetitions] = step 
                if winner(league, newState)[1]:
                    Q[stateMoveTuple(state, move)] = 1 #Reinforce 1 if blue is the winner.
                else:
                    Q[stateMoveTuple(state, move)] = 0 #Reinforce 0 if red is the winner.
            
            if step > 1:
                #print("step>1: " + str(stateMoveTuple(stateOld, moveOld)))
                #print("Q[compMoveTuple(compOld, moveOld)]: " + str(Q[stateMoveTuple(stateOld, moveOld)]))
                #print("Q[compMoveTuple(compNew, move)]: " + str(Q[stateMoveTuple(state, move)]))
                #print("Q[compMoveTuple(compOld, moveOld)]: " + str(Q[stateMoveTuple(stateOld, moveOld)]))
                Q[stateMoveTuple(stateOld, moveOld)] += rho * (Q[stateMoveTuple(state, move)] - Q[stateMoveTuple(stateOld, moveOld)])
                
            stateOld, moveOld = state, move # remember state and move to Q(state,move) can be updated after next steps
            state = newState

    # Returns Q and list or array of number of steps to reach goal for each repetition.
    return Q, outcomes

A "quick" test

In [294]:
nRepetitions = 200
learningRate = 0.5
epsilonDecayFactor = 0.9999
Q, outcomes = trainQ(nRepetitions, learningRate, epsilonDecayFactor, L, getValidChampMoves, getValidBanMoves, makeChampMove)

Game: 0
Game: 1
Game: 2
Game: 3
Game: 4
Game: 5
Game: 6
Game: 7
Game: 8
Game: 9
Game: 10
Game: 11
Game: 12
Game: 13
Game: 14
Game: 15
Game: 16
Game: 17
Game: 18
Game: 19
Game: 20
Game: 21
Game: 22
Game: 23
Game: 24
Game: 25
Game: 26
Game: 27
Game: 28
Game: 29
Game: 30
Game: 31
Game: 32
Game: 33
Game: 34
Game: 35
Game: 36
Game: 37
Game: 38
Game: 39
Game: 40
Game: 41
Game: 42
Game: 43
Game: 44
Game: 45
Game: 46
Game: 47
Game: 48
Game: 49
Game: 50
Game: 51
Game: 52
Game: 53
Game: 54
Game: 55
Game: 56
Game: 57
Game: 58
Game: 59
Game: 60
Game: 61
Game: 62
Game: 63
Game: 64
Game: 65
Game: 66
Game: 67
Game: 68
Game: 69
Game: 70
Game: 71
Game: 72
Game: 73
Game: 74
Game: 75
Game: 76
Game: 77
Game: 78
Game: 79
Game: 80
Game: 81
Game: 82
Game: 83
Game: 84
Game: 85
Game: 86
Game: 87
Game: 88
Game: 89
Game: 90
Game: 91
Game: 92
Game: 93
Game: 94
Game: 95
Game: 96
Game: 97
Game: 98
Game: 99
Game: 100
Game: 101
Game: 102
Game: 103
Game: 104
Game: 105
Game: 106
Game: 107
Game: 108
Game: 109
Game: 110


### Test Q

The way my test Q function is written is nothing special. Each player takes a turn that hopefully benefits them (the turn is taken from perspective of either blue or red, as you can see in the epsillonChampGreedy line). Basically it just plays the game Champ Select, and then gives us the ending teams.

In [284]:
def testQ(Q, league, validMovesF, makeMoveF, bans=[]):
    comp1 = {}
    comp2 = {}
    state = {'blue': comp1, 'red': comp2}
    path = [state]
    for turn in range(11):
        #If blue/comp1 turn [1,4,5,8,9]
        if turn+1 == 1 or turn+1 == 4 or turn+1 == 5 or turn+1 == 8 or turn+1 == 9:

            # Make a move
            move = epsilonChampGreedy(0, Q, league, state['blue'], state['red'], bans)
            newComp = makeChampMove(league, move, state['blue'], state['red'], bans=[])
            state['blue'] = newComp #Update the blue comp


        #If red/comp2 turn [2,3,6,7,10]
        elif turn+1 == 2 or turn+1 == 3 or turn+1 == 6 or turn+1 == 7 or turn+1 == 10: 

            # Make a move
            move = epsilonChampGreedy(0, Q, league, state['red'], state['blue'], bans)
            newComp = makeChampMove(league, move, state['red'], state['blue'], bans=[])
            state['red'] = newComp #Update the red comp
            
        turn += 1
        if winner(league, state)[0]:
            #add to state to end of path
            return path
        
    return path

# Results

Unfortunately, my experiment failed mostly because of time issues. The problem space seems to simply be too large for what I was trying to do given my expectations. I simply couldn't run enough iterations to show that it was functioning.

In [307]:
end = testQ(Q, L, getValidChampMoves, makeChampMove)
end

[{'blue': {'TOP': Aatrox, 266: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
   'JUNGLE': Alistar, 12: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_SUPPORT'],
   'MIDDLE': Amumu, 32: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_SUPPORT'],
   'DUO_CARRY': Ashe, 22: ['MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
   'DUO_SUPPORT': Bard, 432: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']},
  'red': {'TOP': Ahri, 103: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
   'JUNGLE': Akali, 84: ['TOP', 'JUNGLE', 'MIDDLE', 'DUO_CARRY'],
   'MIDDLE': Anivia, 34: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
   'DUO_CARRY': Annie, 1: ['TOP', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT'],
   'DUO_SUPPORT': Blitzcrank, 53: ['JUNGLE', 'MIDDLE', 'DUO_CARRY', 'DUO_SUPPORT']}}]

Let's at least see which team won.

In [309]:
blue = end[0]['blue']
red = end[0]['red']
printState(blue, red, [])

Bans: []
----------------------
TOP
Aatrox vs Ahri

JUNGLE
Alistar vs Akali

MIDDLE
Amumu vs Anivia

DUO_CARRY
Ashe vs Annie

DUO_SUPPORT
Bard vs Blitzcrank

Left team has 0.47 chance of a win.
----------------------


As you can see, the AI didn't learn anything! We always come up with the same composition since the Q table is so ridiculously sparse. The greedy choice is not even able to be taken, as we can see it simply chooses things from valid moves based on alphabetical ordering, only subject to role restrictions.

## My questions from the proposal:
   - "Can I write a program to break the League champion select meta?"
   I think that this program could come up with some seriously interesting compositions It would purely choose things based on what the numbers told it, rather than its perception of reality or the meta. That beign said, it simply was too inefficiently implemented for how large of a problem it is, so I *didn't* write a program that can break the meta by anything but pure chance.

   - "Can I write a program to win more games?" 
   Funnily enough, the "alphabetical comp" it came up with is quite awful from my point of view. Perhaps I need to step out of my own comfort zone and try that comp over and over. Who knows, *maybe* I'll win more games. 
   
   In all seriousness, though, the program won't help a player win more games.
   
   - "Can I predict which team will win in a championship?"
   This isn't something I ended up aiming for in my final project. That being said, calling the correct functions would give a bit of a stat analysis and could possibly give an inkling towards which team would win. But, it doesn't take into account the ridiculously high skill of the championship players or their pool of champions. It look at stats for the general populace, not for the top 0.00000000001% of players.

# Conclusions

Something I learned that is very important about this project is that my problem space is enormous, and the amount of space I used to represent each part of it, as well as some of my algorithms, made for a VERY slow game. What I mean is, every repetition in the repetitions loop in trainQ takes a noticable amount of time (split second, but the eye catches it). Rather than doing hundreds a second, it does a few. If I were to do this again, I would rethink my design and especially rethink some of my data structure choices. I chalk it up, primarily, to inexperience in Python and to be frank, not giving myself enough time. Without doing thousands of computations, I cannot stand behind any of the data I've accrued. That being said, I learned a lot and genuinely want to continue this project as it is something that I think a *lot* of people in my community would enjoy. I think I am really close to something cool, but as it stands I would need to let my code run for a day or more to really get an AI that understood what was doing on.

I needed to be able to run even more repetitions than in tic-tac-toe, but couldn't even match up to 50000 in a reasonable amount of time. This project would need a large overhaul, but that doesn't mean nothing can be learned from it! At least for myself.

It was very difficult for me to get the API's working, as I have never done something like that. Working in Python can be a struggle for me since the vast majority of my programming experience comes from Java, and I was programming two other Java programs while writing this one. I also wrote the program *in this notebook*, so I got bogged down on weird issues that I otherwise wouldn't have had I set it up in PyCharm or in a text editor. 

This project was actually super fun, and I hope to use these resources in the future for something similar to this!


Reader,

I hope you enjoyed reading, and have a fantastic winter break!

### Word count

In [311]:
import io
from IPython.nbformat import current
import glob
nbfile = glob.glob('Pecquet-SemesterProject.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Pecquet-SemesterProject.ipynb is 3580
